In [247]:
import os

In [248]:
%pwd

'C:\\Project\\Text-Summarizer-Project'

In [249]:
%cd C:\Project\Text-Summarizer-Project\

C:\Project\Text-Summarizer-Project


In [250]:
%pwd

'C:\\Project\\Text-Summarizer-Project'

In [251]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [252]:
from Text_Summarizer.constants import *
from Text_Summarizer.utils.common import read_yaml, create_directories

In [253]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [254]:
import os
import urllib.request as request
import zipfile
from Text_Summarizer.logging import logger
from Text_Summarizer.utils.common import get_size

In [255]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [256]:
import gdown

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading file from Google Drive: {self.config.source_URL}")
            gdown.download(self.config.source_URL, str(self.config.local_data_file), quiet=False)
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [257]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-10-04 18:39:25,219: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-04 18:39:25,223: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-04 18:39:25,224: INFO: common: created directory at: artifacts]
[2025-10-04 18:39:25,226: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-04 18:39:25,227: INFO: 3562691708: File already exists of size: ~ 23073 KB]
